In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://westin.marriott.com/hotel-locations/"
driver.get(url)
time.sleep(30)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'property'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
hotelsurl_df = pd.DataFrame(hotel_list)
hotelsurl_df

,name,url
0,"The Westin Cairo Golf Resort & Spa, Katameya D...",https://www.marriott.com/CAIWI
1,"The Westin Turtle Bay Resort & Spa, Mauritius",https://www.marriott.com/MRUTB
2,The Westin Cape Town,https://www.marriott.com/CPTWI
3,The Westin Dhaka,https://www.marriott.com/DACWI
4,The Westin Hefei Baohe,https://www.marriott.com/HFEWI
...,...,...
236,The Westin Virginia Beach Town Center,https://www.marriott.com/ORFWI
237,The Westin Bellevue,https://www.marriott.com/BVUBW
238,The Westin Seattle,https://www.marriott.com/SEAWI
239,The Westin Milwaukee,https://www.marriott.com/MKEIW


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in hotelsurl_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Westin'
        except:
            hotel['brand'] = 'Westin'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df     

{'brand': 'Westin', 'name': 'The Westin Cairo Golf Resort & Spa, Katameya Dunes', 'url': 'https://www.marriott.com/CAIWI', 'street': 'Road 90,New Cairo City, Katameya Dunes', 'locality': 'Cairo', 'state': '', 'postalcode': '11835', 'coordinate1': ' View Rates ', 'coordinate2': '', 'coordinate3': ' View Rates ', 'coordinate4': 'https://www.marriott.com/hotels/maps/travel/caiwi-the-westin-cairo-golf-resort-and-spa-katameya-dunes/?directPageRequest=true', 'description': "[ The Westin Cairo Golf Resort &amp; Spa, Katameya Dunes is a jewel in the center of a prestigious private enclave in New Cairo, Egypt, featuring a pampering 9,000-square-foot spa and a 27-hole golf course designed by Sir Nick Faldo and Brian Curley. The hotel's well-appointed guest rooms and suites provide breathtaking views of tranquil fairways. Each accommodation is a haven for wellness and relaxation, which are ensured by the world-renowned Heavenly® Bed and the reinvigorating Heavenly® Bath experience. An all-day res

{'brand': 'Westin', 'name': 'The Westin Beijing Chaoyang', 'url': 'https://www.marriott.com/BJSWI', 'street': '7 North Dongsanhuan Road, Chaoyang District', 'locality': 'Beijing', 'state': '', 'postalcode': '100027', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/bjswi-the-westin-beijing-chaoyang/?directPageRequest=true', 'coordinate3': '39.9503305', 'coordinate4': '116.4608037', 'description': "[ Ideally situated just 25 minutes from Beijing Capital International Airport and near the city's popular Sanlitun entertainment area, The Westin Beijing Chaoyang places you in the midst of a thriving district of office towers and embassies, international boutiques, restaurants and clubs. Find reprieve and renewal in tranquil hotel rooms featuring intuitive amenities and our signature Westin Heavenly® Bed. Discover culinary delights and moments to savor in one of our six inviting dining options, including Chinese cuisine at Zen5es, succulent meat and seafo

{'brand': 'Westin', 'name': 'The Westin Guangzhou', 'url': 'https://www.marriott.com/CANGW', 'street': '6 Lin He Zhong Road, Tian He District', 'locality': 'Guangzhou', 'state': '', 'postalcode': '510610', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': '[ In heart of the Tian He business district, our hotel is adjacent to CITIC Plaza and China Shine, home to the majority of the Fortune 500 companies in Guangzhou. And our ideal City Center location puts you just steps from high-end shopping and the Guangzhou East Train Station. Stay fit at our WestinWORKOUT® Fitness Studio and in resplendent outdoor and indoor pools or reenergize with a sumptuous treatment to enhance your health at our Heavenly Spa by Westin™. Enjoy delicious nourishment in our five chic restaurants and lounges, which feature a variety of cuisines including Italian and Latin. Our 444 hotel rooms and suites all offer the Heavenly® Bed, high ceilin

{'brand': 'Westin', 'name': 'The Westin Sanya Haitang Bay Resort', 'url': 'https://www.marriott.com/SYXBW', 'street': 'No. 116, Haitang North Road, Haitang Bay', 'locality': 'Sanya', 'state': '', 'postalcode': '572013', 'coordinate1': ' View Rates ', 'coordinate2': '', 'coordinate3': ' View Rates ', 'coordinate4': 'https://www.marriott.com/hotels/maps/travel/syxbw-the-westin-sanya-haitang-bay-resort/?directPageRequest=true', 'description': "[ The Westin Sanya Haitang Bay Resort is a beach hotel located on mainland China's Haitang Island. Find our resort next to China’s biggest duty-free shopping mall, just steps away from the white sandy beaches of Haitang Bay. Featuring 448 inspiring guest rooms, suites and villas amidst 12,000 square meters of tropical waterscapes, our hotel includes swimming pools, whirlpool and fire-torched waterscapes that vary in size and shape. The Westin Sanya Haitang Bay Resort provides a variety of epicurean food and beverage experiences at our on-site restau

{'brand': 'Westin', 'name': 'The Westin Qingdao', 'url': 'https://www.marriott.com/TAOWI', 'street': '8 Hong Kong Middle Road, Shinan District', 'locality': 'Qingdao', 'state': '', 'postalcode': '266071', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/taowi-the-westin-qingdao/?directPageRequest=true', 'coordinate2': '36.0637184', 'coordinate3': '120.3801746', 'coordinate4': 'N/A', 'description': "[ Rise above the fray and find all you need for an empowering stay in Qingdao, China. The Westin Qingdao is a wellness-focused hotel with modern accommodations, delicious dining and a convenient location in the urban center of one of China's fast-rising cities. Step into intuitive rooms and suites designed to keep you fresh and energized during your stay - and inspired by our high-rise hotel's soaring views of the Qingdao city center. Enjoy healthful, authentic dining at our hotel restaurants, from fresh seafood and traditional Chinese to live cooking stations at our breakfast buf

...

{'brand': 'Westin', 'name': 'The Westin Dallas Fort Worth Airport', 'url': 'https://www.marriott.com/DFWWA', 'street': '4545 W. John Carpenter Freeway', 'locality': 'Irving', 'state': 'Texas', 'postalcode': '75063', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/dfwwa-the-westin-dallas-fort-worth-airport/?directPageRequest=true', 'coordinate3': '32.916499', 'coordinate4': '-97.008257', 'description': "[ Settled deep in the heart of Texas, The Westin Dallas Fort Worth Airport welcomes you to a refreshing getaway. The moment you depart Dallas Fort Worth International Airport in our complimentary shuttle service, you'll be enamored by our thoughtful amenities. Find your sanctuary in our tranquil, pet-friendly rooms where luxurious bedding, flat-panel TVs, high-speed Wi-Fi and in-room spa services will help you switch paces and relax. From a coffee house to a casually elegant restaurant, our hotel offers an array of dining options to satisfy your hung

{'brand': 'Westin', 'name': 'The Westin Arlington Gateway', 'url': 'https://www.marriott.com/WASAG', 'street': '801 North Glebe Road', 'locality': 'Arlington', 'state': 'Virginia', 'postalcode': '22203', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/wasag-the-westin-arlington-gateway/?directPageRequest=true', 'coordinate3': '38.880201', 'coordinate4': '-77.114198', 'description': "[ The Westin Arlington Gateway is your dynamic retreat in the vibrant Ballston neighborhood, walking distance from the Ballston-MU Metro station. Our hotel is also a 20-minute drive from Washington D.C.'s Union Station and near a collection of attractions like The Pentagon, Kennedy Center, National Science Foundation and Ronald Reagan Washington National Airport. A variety of holistic hotel amenities includes our on-site Westin WORKOUT® Fitness Studio complete with cardio equipment, free weights and an indoor heated pool.  Modern hotel rooms and spacious suites present 

{'brand': 'Westin', 'name': 'The Westin Richmond', 'url': 'https://www.marriott.com/RICWI', 'street': '6631 West Broad Street', 'locality': 'Richmond', 'state': 'Virginia', 'postalcode': '23230', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/ricwi-the-westin-richmond/?directPageRequest=true', 'coordinate3': '37.604151', 'coordinate4': '-77.517106', 'description': "[ Treat yourself to a relaxing, refreshing stay at The Westin Richmond, located in the heart of the city near West End and Laurel Park. Culture and exceptional beauty are just a short drive away at the Virginia Museum of Fine Art, Agecroft Hall &amp; Gardens and Maymont Park. The University of Richmond's sprawling campus is also convenient to our award-winning hotel. The Westin Richmond offers holistic and thoughtful amenities not often seen in modern hotels. Dine on fresh, organic, natural cuisine at Small Batch Local Kitchen after a trip to the Guinness Brewery. Exercise and re-energi

{'brand': 'Westin', 'name': 'The Westin Lima Hotel & Convention Center', 'url': 'https://www.marriott.com/LIMWI', 'street': 'Calle Las Begonias 450. San Isidro', 'locality': 'Lima', 'state': '', 'postalcode': '27', 'coordinate1': ' View Rates ', 'coordinate2': '', 'coordinate3': ' View Rates ', 'coordinate4': 'https://www.marriott.com/hotels/maps/travel/limwi-the-westin-lima-hotel-and-convention-center/?directPageRequest=true', 'description': "[ Located in Lima's most exclusive and modern district, surrounded by a selection of upscale restaurants, shopping galleries, and beautiful parks; The Westin Lima Hotel, with its spacious rooms, is the perfect place to enjoy a breathtaking view of the city and to recharge energy at our world-famous Heavenly® Bed. Try our delicious breakfast buffet at Market 770™, celebrate a special occasion at our fine-dining Peruvian Restaurant Maras™, or enjoy a renewal experience at Heavenly Spa by Westin™; the largest urban spa in South America, featuring an

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Westin,"The Westin Cairo Golf Resort & Spa, Katameya D...",https://www.marriott.com/CAIWI,"Road 90,New Cairo City, Katameya Dunes",Cairo,,11835,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/ca...,"[ The Westin Cairo Golf Resort &amp; Spa, Kata...",N/A,N/A,N/A
1,Westin,"The Westin Turtle Bay Resort & Spa, Mauritius",https://www.marriott.com/MRUTB,Balaclava,Turtle Bay,,,Reviews,View Rates,,View Rates,"[ Revel in the extraordinary views, superb ser...",N/A,N/A,N/A
2,Westin,The Westin Cape Town,https://www.marriott.com/CPTWI,"Convention Square, Lower Long Street",Cape Town,,8000,Reviews,View Rates,,View Rates,[ Memorable experiences await at The Westin Ca...,N/A,N/A,N/A
3,Westin,The Westin Dhaka,https://www.marriott.com/DACWI,"Main Gulshan Avenue, Plot-01, Road 45, Gulshan-2",Dhaka,,1212,Reviews,https://www.marriott.com/hotels/maps/travel/da...,23.79304,90.414341,"[ Discover Dhaka, Bangladesh, from The Westin ...",N/A,4.4,4.5
4,Westin,The Westin Hefei Baohe,https://www.marriott.com/HFEWI,"No. 150 Ma'An Shan Road, Baohe District",Hefei,,230011,Reviews,https://www.marriott.com/hotels/maps/travel/hf...,31.8588999999999,117.3018,[ A meaningful experience awaits you at The We...,N/A,4.5,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,Westin,The Westin Virginia Beach Town Center,https://www.marriott.com/ORFWI,4535 Commerce Street,Virginia Beach,Virginia,23462,Reviews,https://www.marriott.com/hotels/maps/travel/or...,36.841821,-76.135215,[ Soar above it all at The Westin Virginia Bea...,N/A,4.2,4.0
237,Westin,The Westin Bellevue,https://www.marriott.com/BVUBW,600 Bellevue Way NE,Bellevue,Washington,98004,,View Rates,https://www.marriott.com/hotels/maps/travel/bv...,47.6156,[ Prioritize your well-being at The Westin Bel...,N/A,N/A,N/A
238,Westin,The Westin Seattle,https://www.marriott.com/SEAWI,1900 5th Avenue,Seattle,Washington,98101,Reviews,View Rates,,View Rates,[ One of downtown Seattle's most dynamic desti...,N/A,N/A,N/A
239,Westin,The Westin Milwaukee,https://www.marriott.com/MKEIW,550 North Van Buren Street,Milwaukee,Wisconsin,53202,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/mk...,"[ Located right in the heart of downtown, The ...",N/A,N/A,N/A


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Westin,"The Westin Cairo Golf Resort & Spa, Katameya D...",https://www.marriott.com/CAIWI,"Road 90,New Cairo City, Katameya Dunes",Cairo,,11835,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/ca...,"[ The Westin Cairo Golf Resort &amp; Spa, Kata...",N/A,N/A,N/A
1,Westin,"The Westin Turtle Bay Resort & Spa, Mauritius",https://www.marriott.com/MRUTB,Balaclava,Turtle Bay,,,Reviews,View Rates,,View Rates,"[ Revel in the extraordinary views, superb ser...",N/A,N/A,N/A
2,Westin,The Westin Cape Town,https://www.marriott.com/CPTWI,"Convention Square, Lower Long Street",Cape Town,,8000,Reviews,View Rates,,View Rates,[ Memorable experiences await at The Westin Ca...,N/A,N/A,N/A
3,Westin,The Westin Dhaka,https://www.marriott.com/DACWI,"Main Gulshan Avenue, Plot-01, Road 45, Gulshan-2",Dhaka,,1212,Reviews,https://www.marriott.com/hotels/maps/travel/da...,23.79304,90.414341,"[ Discover Dhaka, Bangladesh, from The Westin ...",N/A,4.4,4.5
4,Westin,The Westin Hefei Baohe,https://www.marriott.com/HFEWI,"No. 150 Ma'An Shan Road, Baohe District",Hefei,,230011,Reviews,https://www.marriott.com/hotels/maps/travel/hf...,31.8588999999999,117.3018,[ A meaningful experience awaits you at The We...,N/A,4.5,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,Westin,The Westin Virginia Beach Town Center,https://www.marriott.com/ORFWI,4535 Commerce Street,Virginia Beach,Virginia,23462,Reviews,https://www.marriott.com/hotels/maps/travel/or...,36.841821,-76.135215,[ Soar above it all at The Westin Virginia Bea...,N/A,4.2,4.0
237,Westin,The Westin Bellevue,https://www.marriott.com/BVUBW,600 Bellevue Way NE,Bellevue,Washington,98004,,View Rates,https://www.marriott.com/hotels/maps/travel/bv...,47.6156,[ Prioritize your well-being at The Westin Bel...,N/A,N/A,N/A
238,Westin,The Westin Seattle,https://www.marriott.com/SEAWI,1900 5th Avenue,Seattle,Washington,98101,Reviews,View Rates,,View Rates,[ One of downtown Seattle's most dynamic desti...,N/A,N/A,N/A
239,Westin,The Westin Milwaukee,https://www.marriott.com/MKEIW,550 North Van Buren Street,Milwaukee,Wisconsin,53202,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/mk...,"[ Located right in the heart of downtown, The ...",N/A,N/A,N/A


In [8]:
hotel_df.to_csv('westin_description.csv')